In [ ]:
# ✅ 설정 및 라이브러리 로드
import os
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import pandas as pd
from datetime import datetime


In [ ]:
# ✅ 모델 불러오기
pipe_txt2img = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16
).to("cuda")

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16
).to("cuda")


In [ ]:
# ✅ 업로드 및 전처리 함수
def preprocess_image(image):
    return image.resize((512, 512))

def generate(prompt, image=None, strength=0.75):
    if image:
        image = preprocess_image(image)
        output = pipe_img2img(prompt=prompt, init_image=image, strength=strength, guidance_scale=7.5).images[0]
    else:
        output = pipe_txt2img(prompt=prompt, guidance_scale=7.5).images[0]

    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    output_path = f"output_{timestamp.replace(':','-')}.png"
    output.save(output_path)

    # 로그 기록
    if os.path.exists("log.csv"):
        df = pd.read_csv("log.csv")
    else:
        df = pd.DataFrame(columns=["timestamp", "prompt", "output_path", "guidance_scale"])

    df.loc[len(df.index)] = [timestamp, prompt, output_path, 7.5]
    df.to_csv("log.csv", index=False)

    return output


In [ ]:
# ✅ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### 🎨 Stable Diffusion LoRA 자동 생성기")
    prompt = gr.Textbox(label="프롬프트 입력", placeholder="쿠션을 사용하는 장면...")
    image = gr.Image(label="옵션: 기존 이미지", type="pil")
    strength = gr.Slider(minimum=0.1, maximum=1.0, value=0.75, label="강도 (img2img 전용)")
    output = gr.Image(label="결과 이미지")

    btn = gr.Button("이미지 생성")
    btn.click(fn=generate, inputs=[prompt, image, strength], outputs=[output])

demo.launch(debug=True)


In [ ]:
# ✅ requirements.txt
with open("requirements.txt", "w") as f:
    f.write("""diffusers==0.26.3
transformers==4.41.1
huggingface_hub==0.25.1
accelerate==0.29.2
torch
Pillow
gradio==3.50.2
peft==0.10.0
torchvision
ipywidgets
""")

# ✅ .gitignore
with open(".gitignore", "w") as f:
    f.write("""__pycache__/
*.pyc
*.pkl
output/
lora_training/
trained_lora_model/
.env
""")

# ✅ GitHub Actions 워크플로우
os.makedirs(".github/workflows", exist_ok=True)
with open(".github/workflows/test.yml", "w") as f:
    f.write("""name: Test Notebook

on:
  push:
    branches: [ main ]

jobs:
  run-notebook:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.11'
      - name: Install dependencies
        run: |
          pip install -r requirements.txt
      - name: Run Notebook (check syntax)
        uses: yaananth/run-ipynb@v1
        with:
          path: notebooks/auto_train_lora.ipynb
""")

# ✅ log.csv 생성
if not os.path.exists("log.csv"):
    pd.DataFrame(columns=["timestamp", "prompt", "output_path", "guidance_scale"]).to_csv("log.csv", index=False)
